# Global Variables

In [96]:
from pathlib import Path
# coding: utf-8

exam_name = "2024_YKS_AYT"

# --- Example Usage ---
PROJECT_ID = "lm-harness-question-prepare"  # <--- Replace with your Project ID
LOCATION = "us-central1"         # <--- Replace with your desired region


doc_path = Path("..") /   "data_raw" / f"{exam_name}.pdf"

image_folder = Path("..") / "image_dump" / exam_name


exam_year = exam_name.split("_")[0]
exam_type = exam_name.split("_")[1]
exam_subtype = exam_name.split("_")[2]

start_page_num = 1

answer_page_num = 42 


### Dump all the images to see how I can format them


In [86]:
import pymupdf
from pathlib import Path
from PIL import Image
import io




doc_path = Path("..") /   "data_raw" / f"{exam_name}.pdf"

doc = pymupdf.open(doc_path)

# Create the directory to save images if it doesn't exist


image_folder.mkdir(parents=True, exist_ok=True)

for page_num in range(start_page_num, answer_page_num):

    print(f"Page {page_num}")

    page = doc[page_num]

    image_list = page.get_images(full=True)
    
    for img_index, img in enumerate(image_list):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]

        image = Image.open(io.BytesIO(image_bytes))

        # Save the image to a file
        image_path = Path(image_folder) / f"page_{page_num}_img_{img_index}.{image_ext}"
        image.save(image_path)


Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43


IndexError: page 43 not in document

# Basic Data Dictionary


Question_Object
- Year
- Exam
- SubExam
- Section
- QuestionNumber
- Page
- ImageCount
- HasImage
- MaxImageWidth
- MaxImageHeight
- Passage
- Question
- Choices
- Q_Language
- Answer
- AnswerKey

Image_Object
- Year
- Exam
- Section
- QuestionNumber
- Page
- ImageHeight
- ImageWidth
- ImageOCR
- T_Language
- ImageDescription
- D_Language

# Helper Methods


### To check if something is the begging of a question


### Extract the question number


### Split the question and the choices


In [42]:
import re



# Function to check if a string is in the form of a number
def is_question_index(text):
    # Define the regex pattern
    pattern = r'^\d+\.$'
    return bool(re.match(pattern, text))

def extract_numbers(text):
    # Regular expression to extract the first two numbers

    pattern = r'\b(\d+)\b'

    numbers = re.findall(pattern, text)

    return numbers


def split_question_choices(text):

    pattern = r'\?\s*[\r\n]*\s*'


    # Split the text into parts based on the pattern
    parts = re.split(pattern, text)
    
    return parts


def has_choices(text, expected_choices=5):

    pattern = r'[A-E]\)'

    return expected_choices == len(re.findall(pattern, text))

def turkish_to_ascii(text):
    turkish_chars = "çğıöşüÇĞİÖŞÜ"
    ascii_chars = "cgiosuCGIOSU"
    translation_table = str.maketrans(turkish_chars, ascii_chars)
    return text.translate(translation_table)



# Road Map


1- Split the page into two <br>
2- Sort all the elements within the right and left. <br>
3- Find the beginning of the questions <br>


In [87]:
import pymupdf
from pathlib import Path

#Json 
import json


doc = pymupdf.open(doc_path)

question_list = []
current_section = ""

multi_context_question_text = ""
multi_context_question_set = []
multi_context = False
sections = []
answer_key = False

for page_num in range(len(doc)):
        
        page = doc.load_page(page_num)  

        # Extract words as tuples (x0, y0, x1, y1, "text", block_no, line_no, word_no)
        blocks = page.get_text("blocks")

        right_blocks = []
        left_blocks = []

        page_width = page.rect.width

        for block in blocks:
            if block[0] < page_width / 2:
                # Block is on the left side of the page
                my_text_block= (block[0], block[1], block[2], block[3], block[4], block[5], block[6], 0)
                left_blocks.append(my_text_block)
            else:
                my_text_block= (block[0], block[1], block[2], block[3], block[4], block[5], block[6], 0)
                right_blocks.append(my_text_block)



        image_list = page.get_images(full=True)

        for image_ind, img_info in enumerate(image_list):
            xref = img_info[0]
            
            positions = page.get_image_rects(xref)
            

            height = positions[0][3] - positions[0][1]
            width = positions[0][2] - positions[0][0]

            base_image = doc.extract_image(xref)
            image_ext = base_image["ext"]

            my_img_obj = (positions[0][0], positions[0][1], positions[0][2], positions[0][3], xref, (height, width, image_ext), image_ind, 1)

            if my_img_obj[0] < page_width / 2:
                left_blocks.append(my_img_obj)
            else:
                right_blocks.append(my_img_obj)


        # Sort the blocks by their y-coordinates (top to bottom) and then by x-coordinates (left to right)
        left_blocks.sort(key=lambda x: (x[1], x[0]))
        right_blocks.sort(key=lambda x: (x[1], x[0]))

        # Combine the sorted blocks into a single list
        combined_blocks = left_blocks + right_blocks

        
        current_question = {}

        if not answer_key:
                for block in combined_blocks:
                    if block[7] == 0:
                        block_text = block[4]

                        #Check if the block is empty
                        if block_text.strip() == "":
                            continue

                        # If it is the beginning of a new chapter (correct way to check multiple strings)
                        if any(phrase in block_text for phrase in ["1. Bu testte", "2. Cevaplarınızı", "Ö S Y M", 
                            "Diğer sayfaya geçiniz.", "Bu soruların telif hakları", "hiçbir kişi, kurum veya kuruluş", "soruları Din Kültürü ve Ahlak Bilgisi", "TESTİ BİTTİ."]):
                            continue

                        if "CEVAPLARINIZI KONTROL EDİNİZ" in block_text:
                            # If it's the answer key, set the flag to True
                            answer_key = True
                            continue

                        if "soruları aşağıdaki parçaya göre" in block_text:
                            # If it's a multi-context question, save the text and continue
                            current_question = {}
                            multi_context = True
                            indeces = extract_numbers(block_text)
                            sub_index = int(indeces[0])
                            upper_index = int(indeces[-1])
                            multi_context_question_set = [index for index in range(sub_index, upper_index + 1)]
                            continue
                        if multi_context:
                            # If it's a multi-context question, save the text and continue
                            if multi_context_question_text == "":
                                multi_context_question_text = block_text
                            else:   
                                multi_context_question_text += " " + block_text
                            


                        if f"{exam_year}-{exam_subtype}/" in block_text:
                            if "\n" in block_text and len(block_text.split("\n")) > 1:
                                if block_text.split("\n")[1] != "":
                                    current_section = turkish_to_ascii(block_text.split("\n")[1].strip().replace(" ", "-"))
                                    sections.append(current_section)
                            continue
                        
                        if is_question_index(block_text):
                            # Close the multi-context question if it was open
                            if multi_context:
                                multi_context = False



                            # If it's a new question, create a new dictionary
                        
                            new_question = {}
                            new_question["year"] = exam_year
                            new_question["exam"] = exam_type
                            new_question["subtype"] = exam_subtype
                            new_question["section"] = current_section
                            new_question["question_number"] = int(block_text.split(".")[0])
                            new_question["page"] = page_num
                            new_question["q_language"] = "tr"
                            new_question["question"] = ""
                            new_question["has_image"] = False
                            new_question["max_image_heigth"] = 0
                            new_question["max_image_width"] = 0
                            new_question["passage"] = ""
                            new_question["choices"] = []
                            new_question["answer"] = ""
                            new_question["image_count"] = 0
                            new_question["answer_key"] = False
                            current_question = new_question
                            question_list.append(current_question)

                            if multi_context_question_text != "":
                                # If there was a multi-context question, add it to the current question
                                if new_question["question_number"] in multi_context_question_set:
                                    current_question["passage"] = multi_context_question_text
                                    if new_question["question_number"] == multi_context_question_set[-1]:
                                        multi_context_question_text = ""
                                        multi_context_question_set = []


                            continue

                        if current_question:
                            # Append the text to the current question - FIXED HERE
                            if current_question["question"] == "":
                                current_question["question"] = block_text  # Actually save the first text block
                            else:
                                current_question["question"] += " " + block_text

                    elif block[7] == 1:
                        # If it's an image block, add the image reference to the current question
                        if current_question:
                            current_question["has_image"] = True
                            current_question["image_count"] += 1
                            current_question["max_image_heigth"] = max(current_question["max_image_heigth"], block[5][0])
                            current_question["max_image_width"] = max(current_question["max_image_width"], block[5][1])

                            if current_question["question"] == "":
                                current_question["question"] = f"<img_{block[6]}.{block[5][2]}> "
                            else:
                                current_question["question"] += f" <img_{block[6]}.{block[5][2]}> "
                            # Find the image file and change the name append the section info

                            image_path = Path(image_folder)/ f"page_{page_num}_img_{block[6]}.{block[5][2]}"

                            turkify_name = turkish_to_ascii(current_section.replace(" ", "-"))

                            new_image_path = Path(image_folder) / f"{turkify_name}_page_{page_num}_img_{block[6]}.{block[5][2]}" 

                            image_path.rename(new_image_path)






# Order the choices as they are mixed

# everything after the '?' is a choice

for question in question_list:

    parts = []

    # Split the question text into parts
    parts = split_question_choices(question["question"])
    
    # If there are multiple parts, the first part is the question and the rest are choices
    if len(parts) > 1:
        question_text = parts[0] + "?"
        
        i = 1

        while i < len(parts):
            if has_choices(parts[i], expected_choices=5):

                choices_text = parts[i]
                break
            
            i += 1


        
        # Split the choices by line breaks and remove empty strings
        choices = [choice.strip() for choice in choices_text.split("\n") if choice.strip()]
        
        # Assign the question text and choices to the dictionary
        question["question"] = question_text.strip()
        question["choices_raw"] = choices_text
        question["choices"] = create_choice_list(choices_text)




## Test if the choices extrated correctly

for question in question_list:

    if not has_choices(question["choices_raw"], expected_choices=5):
        print(question["section"], question["question_number"])

    if question["choices"] == -1:
        print(f"Section: {question["section"]}, {question["question_number"]}")

final_version = {
    "exam_year": exam_year,
    "exam_type": exam_type,
    "exam_subtype": exam_subtype,
    "sections": sections,
    "questions": question_list,
    


}

# Primt the question list as json
json_data = json.dumps(final_version, ensure_ascii=False, indent=4)


# Save the JSON data to a file
with open(f"../data/{exam_name}_questions.json", "w", encoding="utf-8") as json_file:
    json_file.write(json_data)


        

            



Choices are not extracted correctly 1
Section: TURKCE-TESTI, 11
Section: SOSYAL-BILIMLER-TESTI, 4
Section: SOSYAL-BILIMLER-TESTI, 17
Section: TEMEL-MATEMATIK-TESTI, 8
Section: TEMEL-MATEMATIK-TESTI, 10
Section: TEMEL-MATEMATIK-TESTI, 13
Section: TEMEL-MATEMATIK-TESTI, 15
Section: TEMEL-MATEMATIK-TESTI, 16
Section: FEN-BILIMLERI-TESTI, 5
Section: FEN-BILIMLERI-TESTI, 8
Section: FEN-BILIMLERI-TESTI, 10


Errors for TYT 2024 Fix Manually

Choices are not extracted correctly 1

Section: TURKCE-TESTI, 11

Section: SOSYAL-BILIMLER-TESTI, 4

Section: SOSYAL-BILIMLER-TESTI, 17

Section: TEMEL-MATEMATIK-TESTI, 8

Section: TEMEL-MATEMATIK-TESTI, 10

Section: TEMEL-MATEMATIK-TESTI, 13

Section: TEMEL-MATEMATIK-TESTI, 15

Section: TEMEL-MATEMATIK-TESTI, 16

Section: FEN-BILIMLERI-TESTI, 5

Section: FEN-BILIMLERI-TESTI, 8

Section: FEN-BILIMLERI-TESTI, 10


Error AYT2024 MANUAL FIX

Section: TURK-DILI-VE-EDEBIYATI-SOSYAL, 10

Section: MATEMATIK-TESTI, 1

Section: MATEMATIK-TESTI, 8

Section: MATEMATIK-TESTI, 9

Section: MATEMATIK-TESTI, 13

Section: MATEMATIK-TESTI, 16

Section: MATEMATIK-TESTI, 17

Section: MATEMATIK-TESTI, 21

Section: MATEMATIK-TESTI, 38

Section: FEN-BILIMLERI-TESTI, 16

In [44]:
import re

def create_choice_list(choices):

        res = re.split(r'\n', choices)


        #remove empty strings
        res = [i for i in res if len(i.strip()) > 0]

        if len(res) < 10:
            res = re.split(r'[\s]', choices)
            #remove empty strings again
            res = [i for i in res if len(i.strip()) > 0]

        elif len(res) > 10:
            #remove empty strings again
            

            new_res = [""] * 10
            new_res_ind = -1
            for index in range(len(res)):
                first_char = res[index].lstrip()[0]
                if first_char.isupper() or first_char in ["<"]:

                    new_res_ind += 1
                    if new_res_ind > 9:
                        #print("\033[31mChoices are not extracted correctly 4\033[0m")
                        return -1
                    new_res[new_res_ind] = res[index]
                else:
                    new_res[new_res_ind] += " " + res[index]
                    
                    

            res = new_res

        
        
        #if there is problem notice the user
        if len(res) != 10:
            print("\033[31mChoices are not extracted correctly 1\033[0m")
            return -1

     
        result = [""] * 10

        choice_ind = 0
        non_choice_ind = 1

        for i in range(10):

            # Check if the string matches the pattern for choices (A), B), C), D), E))

            if bool(re.match(r'\s*[ABCDE]\)', res[i])):
                if choice_ind > 9:
                    #print("\033[31mChoices are not extracted correctly 2\033[0m")
                    return -1
                result[choice_ind] = res[i]
             
                choice_ind += 2
            else:
                if non_choice_ind > 9:
                    #print("\033[31mChoices are not extracted correctly 3\033[0m")
               
                    return -1
                result[non_choice_ind] = res[i]
                
                non_choice_ind += 2

        final_result = [f"{result[i]} {result[i + 1]}" for i in range(0, len(result), 2)]

        return final_result






choices_text = "<page_25_img_6.png>  <page_25_img_5.png>  A)\nB)\n <page_25_img_7.png>  <page_25_img_8.png>  C)\nD)\n <page_25_img_9.png>  E)\n <page_25_img_10.png> "


create_choice_list(choices_text)

Choices are not extracted correctly 1


-1

# Answer Key

In [90]:
# Add the answer key to the questions
import json

from pathlib import Path

import pandas as pd



# Load the JSON data from the file

json_path = Path("..") / "data" / f"{exam_name}_questions.json"

with open(json_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

question_frame = pd.DataFrame(data["questions"])



doc = pymupdf.open(doc_path)



answer_page = doc[answer_page_num]

blocks = answer_page.get_text("blocks")


current_section = ""

for i in blocks:
    print(f">{i[4]}<")
    if i[4].strip() == "":
        continue



    turkified = turkish_to_ascii(i[4].strip().replace(" ", "-").replace(" ", "-"))

    if turkified in data["sections"]:
        print(f"Section: {turkified}")
        current_section = turkified
        continue
    if current_section != "" and i[4].strip() != "":


        # Change the answer in pandas dataframe

        answers = i[4].strip().split("\n")

        for answer in answers:
            question_number = int(answer.split(".")[0])
            question_answer = answer.split(".")[1].strip()



            # look for the coorect section in the dataframe and the correct question number

            question_frame.loc[(question_frame["section"] == current_section) & (question_frame["question_number"] == int(question_number)), "answer"] = question_answer.strip()
            question_frame.loc[(question_frame["section"] == current_section) & (question_frame["question_number"] == int(question_number)), "answer_key"] = True
        current_section = ""


# Turn the data frame to list of dicts

question_list = question_frame.to_dict(orient="records")

data["questions"] = question_list

# Save the JSON data to a file

with open(f"../data/{exam_name}_questions.json", "w", encoding="utf-8") as json_file:
    # do the json.dumps here

    json_file.write(json.dumps(data, ensure_ascii=False, indent=4))


            




>2024 TEMEL YETERLİLİK TESTİ (TYT)
<
>2024 YKS 1. OTURUM TEMEL YETERLİLİK TESTİ (TYT)
<
>08-06-2024
<
>TÜRKÇE TESTİ
<
Section: TURKCE-TESTI
>1.   B
2.   A
3.   A
4.   D
5.   B
6.   D
7.   B
8.   C
9.   C
10.   C
11.   E
12.   D
13.   C
14.   B
15.   D
16.   D
17.   A
18.   D
19.   C
20.   C
21.   E
22.   B
23.   E
24.   A
25.   B
26.   B
27.   E
28.   D
29.   C
30.   E
31.   C
32.   A
33.   A
34.   D
35.   E
36.   A
37.   D
38.   E
39.   B
40.   A
<
>SOSYAL BİLİMLER TESTİ
<
Section: SOSYAL-BILIMLER-TESTI
>1.   D
2.   E
3.   E
4.   C
5.   B
6.   B
7.   C
8.   A
9.   D
10.   C
11.   B
12.   D
13.   D
14.   E
15.   B
16.   A
17.   C
18.   E
19.   D
20.   A
21.   E
22.   A
23.   C
24.   C
25.   A
<
>TEMEL MATEMATİK TESTİ
<
Section: TEMEL-MATEMATIK-TESTI
>1.   A
2.   E
3.   A
4.   D
5.   C
6.   B
7.   D
8.   A
9.   C
10.   B
11.   E
12.   A
13.   E
14.   B
15.   D
16.   A
17.   C
18.   C
19.   B
20.   D
21.   E
22.   C
23.   B
24.   A
25.   C
26.   B
27.   E
28.   B
29.   E
30.   B
31.   D


For the questions in literature manuel help is required

In [78]:
answers_literature = "1. B\n2. E\n3. B\n4. D\n5. C\n6. D\n7. C\n8. E\n9. B\n10. E\n11. B\n12. A\n13. B\n14. A\n15. E\n16. B\n17. E\n18. D\n19. A\n20. C\n21. D\n22. E\n23. B\n24. C\n25. A\n26. E\n27. C\n28. A\n29. B\n30. D\n31. C\n32. A\n33. E\n34. A\n35. C\n36. A\n37. D\n38. E\n39. B\n40. D"


# Load the JSON data from the file

json_path = Path("..") / "data" / f"{exam_name}_questions.json"

with open(json_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

question_frame = pd.DataFrame(data["questions"])


current_section = "TURK-DILI-VE-EDEBIYATI-SOSYAL"

answers = answers_literature.strip().split("\n")

for answer in answers:
            question_number = int(answer.split(".")[0])
            question_answer = answer.split(".")[1].strip()



            # look for the coorect section in the dataframe and the correct question number

            question_frame.loc[(question_frame["section"] == current_section) & (question_frame["question_number"] == int(question_number)), "answer"] = question_answer.strip()
            question_frame.loc[(question_frame["section"] == current_section) & (question_frame["question_number"] == int(question_number)), "answer_key"] = True



# Turn the data frame to list of dicts

question_list = question_frame.to_dict(orient="records")

data["questions"] = question_list

# Save the JSON data to a file

with open(f"../data/{exam_name}_questions.json", "w", encoding="utf-8") as json_file:
    # do the json.dumps here

    json_file.write(json.dumps(data, ensure_ascii=False, indent=4))


# Use Pandas to proccess the JSON

In [169]:
import pandas as pd
import json

from pathlib import Path

exam_name = "2024_YKS_TYT"

# Load the JSON data from the file

json_path = Path("..") / "data" / f"{exam_name}_questions.json"

with open(json_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Convert the JSON data to a DataFrame

df = pd.DataFrame(data)

df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              125 non-null    object 
 1   exam              125 non-null    object 
 2   subtype           125 non-null    object 
 3   section           125 non-null    object 
 4   question_number   125 non-null    int64  
 5   page              125 non-null    int64  
 6   q_language        125 non-null    object 
 7   question          125 non-null    object 
 8   has_image         125 non-null    bool   
 9   max_image_heigth  125 non-null    float64
 10  max_image_width   125 non-null    float64
 11  passage           125 non-null    object 
 12  choices           125 non-null    object 
 13  answer            125 non-null    object 
 14  image_count       125 non-null    int64  
 15  choices_raw       125 non-null    object 
dtypes: bool(1), float64(2), int64(3), object(10)

# Process the images

Use math OCR

Create an image database

Image_Object
- Year
- Exam
- Section
- QuestionNumber
- Page
- ImageHeight
- ImageWidth
- ImageOCR
- T_Language
- ImageDescription
- D_Language
- Extension

In [97]:
import os

# go over image_dump/exam_name and create an entry for the database




images = []

sections = []


for image_path in image_folder.glob("**/*"):
    if image_path.is_file():

        # get the file name and the path
        file_name = image_path.name

        # Check if it is a question image
        if "_page_" not in file_name:
            #remove the file
            os.remove(image_path)
            continue
        

        # get the file dimesions

        image = Image.open(image_path)
        width, height = image.size
        image.close()

        section = file_name.split("_")[0]
        if section not in sections:
            sections.append(section)

        image_entry = {
            "year": exam_year,
            "exam": exam_type,
            "subtype": exam_subtype,
            "section": section,
            "page": int(file_name.split("_")[2].split(".")[0]),
            "image_height": height,
            "image_width": width,
            "image_number" : int(file_name.split("_")[4].split(".")[0]),
            "image_ocr": "",
            "ocr_language": "",
            "image_description": "",
            "description_language": "",
            "extension": file_name.split(".")[1],
        }

        images.append(image_entry)

# turn everything into json

image_json = {
    "exam_year": exam_year,
    "exam_type": exam_type,
    "exam_subtype": exam_subtype,
    "sections": sections,
    "images": images,

}


# Save the JSON data to a file
with open(f"../data/{exam_name}_images.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(image_json, ensure_ascii=False, indent=4))
        

# Gemini API


In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
import base64 # Required for local file encoding




project_id = PROJECT_ID
location = LOCATION
# Initialize Vertex AI
model_name = "gemini-1.0-pro-vision-001" # Or use gemini-1.5-pro-preview-0409 for the latest preview
#model_name = "gemini-1.5-pro-preview-0409"
vertexai.init(project=project_id, location=location)


model = GenerativeModel(model_name)





def generate_image_description(
    image_source: str, # Can be a local path or a gs:// GCS URI
    prompt: str = "Create the Latex ASCİİ and Mathemtica for this expression", # The instruction for Gemini
    ) -> str:
    """
    Generates a text description for an image using Gemini.

    Args:
        project_id: Your Google Cloud project ID.
        location: The Google Cloud region (e.g., "us-central1").
        image_source: Path to a local image file OR a GCS URI (gs://...).
        prompt: The text prompt to guide the description generation.
        model_name: The specific Gemini model to use.

    Returns:
        The generated image description as a string, or an error message.
    """
    image_part = None

    
    # Process local image file
    try:
            with open(image_source, "rb") as f:
                image_bytes = f.read()
            # Encode to base64 is often preferred by APIs, but SDK might handle bytes directly.
            # Let's create the Part directly from bytes, SDK should handle necessary encoding.
            image_part = Part.from_data(
                mime_type="image/jpeg", # Adjust mime_type if needed (e.g., image/png)
                data=image_bytes
            )
            print("Local image loaded and prepared.")
    except FileNotFoundError:
            return f"Error: Local image file not found at {image_source}"
    except Exception as e:
            return f"Error loading local image: {e}"

    if not image_part:
         return "Error: Could not create image part."

    # Construct the prompt content
    # The order matters: typically text prompt then image(s)
    content = [prompt, image_part]


    # Generation configuration (optional)
    generation_config = {
        "max_output_tokens": 2048,
        "temperature": 0.4, # Lower temperature for more factual descriptions
        "top_p": 1.0,
        "top_k": 32,
    }

    print("Sending request to Gemini API...")
    try:
        responses = model.generate_content(
            content,
            generation_config=generation_config,
            stream=False, # Set to True for streaming response
        )

        # Handling the response
        if responses.candidates:
             # Check finish reason
            if responses.candidates[0].finish_reason == FinishReason.SAFETY:
                return "Error: The response was blocked due to safety concerns."
            elif responses.candidates[0].finish_reason == FinishReason.RECITATION:
                 return "Error: The response was blocked due to potential recitation."
            elif responses.candidates[0].content and responses.candidates[0].content.parts:
                
                description = responses.candidates[0].content.parts[0].text
                print("Successfully received description.")
                return description
            else:
                return "Error: Received an empty response from the API."
        else:
            # Investigate potential blocking or other issues
            # Check response.prompt_feedback for block reasons
            block_reason = responses.prompt_feedback.block_reason if responses.prompt_feedback else "Unknown"
            return f"Error: No candidates returned. Potential block reason: {block_reason}. Full response: {responses}"


    except Exception as e:
        return f"An error occurred during API call: {e}"






In [18]:

local_image_path = "../image_dump/2024_YKS_TYT/page_25_img_4.png" # <--- Replace with path to your image
description_local = generate_image_description(local_image_path)
print("\n--- Description for Local Image ---")
print(description_local)

Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.

--- Description for Local Image ---
 **LaTeX**

\documentclass{article}
\usepackage{tikz}
\begin{document}
\begin{tikzpicture}
\draw (0,0) rectangle (2,10);
\draw (1,0) -- (1,10);
\draw (0,5) -- (2,5);
\draw (0,0) -- (0,10);
\draw (2,0) -- (2,10);
\node at (1,9) {$0\degree C$};
\node at (1,4) {$20$};
\node at (1,-1) {$-10$};
\draw [red, thick] (1,5) -- (1,9);
\draw [red, thick] (1,9) circle (2mm);
\end{tikzpicture}
\end{document}

**ASCII**

```
  __________________
 /                  \
|    0 degree C    |
|                  |
|                  |
|                  |
|                  |
|                  |
|     -10 degree C  |
|__________________|
```

**Mathematica**

```
ThermometerPlot[20]
```


Add captions to every image

In [98]:


def generate_image_descriptions(exam_name):
    
    # Load the JSON data from the file
    images_json = Path("..") / "data" / f"{exam_name}_images.json"


    image_folder = Path("..") / "image_dump" / exam_name

    with open(images_json, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)

    # Convert the JSON data to a DataFrame

    image_frame = pd.DataFrame(data["images"])

    section_prompts = {
        "FEN-BILIMLERI-TESTI" : "This is an image from a science question. If exists provide, the latex and ascii for the expressions. If not, provide a description of the image. Provide anything that would help a blind Large Language Model to understand the question and solve it successuffly.",
            "SOSYAL-BILIMLER-TESTI" : "This is an image from a social science question. If exists provide, describe the historic or geographic objects. Provide a description of the image. Provide anything that would help a blind Large Language Model to understand the question and solve it successuffly.",
            "TEMEL-MATEMATIK-TESTI" : "This is an image from a math question. If exists provide, the latex and ascii for the expressions. If not, provide a description of the image. Provide anything that would help a blind Large Language Model to understand the question and solve it successuffly. For geometric shapes, provide the names of the shapes and their properties in latex and ascii.",
            "MATEMATIK-TESTI" : "This is an image from a math question. If exists provide, the latex and ascii for the expressions. If not, provide a description of the image. Provide anything that would help a blind Large Language Model to understand the question and solve it successuffly. For geometric shapes, provide the names of the shapes and their properties in latex and ascii.",
            "SOSYAL-BILIMLER-2-TESTI": "This is an image from a social science question. If exists provide, describe the historic or geographic objects. Provide a description of the image. Provide anything that would help a blind Large Language Model to understand the question and solve it successuffly.",
            "TURK-DILI-VE-EDEBIYATI-SOSYAL" : "This a question from liturature and social science exam. If exists provide, describe the historic or geographic objects. Provide a description of the image. Provide anything that would help a blind Large Language Model to understand the question and solve it successuffly.",

    }


    # Loop through each image and generate a description

    for index, row in image_frame.iterrows():
        image_path = Path(image_folder) / f"{row['section']}_page_{row['page']}_img_{row["image_number"]}.{row['extension']}"

        # Check if the image file exists
        if not image_path.exists():
            print(f"Image file does not exist: {image_path}")
            continue

        # Get the prompt for the current section
        prompt = section_prompts.get(row["section"], "This is an image from a question. Provide a description of the image for a blind Large Language Model so that it can solve the question successfully .")

        # Generate the description using the function
        description = generate_image_description(image_path, prompt)

        # Update the DataFrame with the generated description
        image_frame.at[index, "image_description"] = description
        image_frame.at[index, "description_language"] = "tr"

    #save the image frame to a json file


    image_json = {
        "exam_year": exam_year,
        "exam_type": exam_type,
        "exam_subtype": exam_subtype,
        "sections": data["sections"],
        "images": image_frame.to_dict(orient="records"),
    }

    # Save the JSON data to a file

    with open(f"../data/{exam_name}_images.json", "w", encoding="utf-8") as json_file:
        json_file.write(json.dumps(image_json, ensure_ascii=False, indent=4))

In [101]:
generate_image_descriptions("2024_YKS_AYT")

Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Suc

In [100]:
generate_image_descriptions("2024_YKS_TYT")


Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Successfully received description.
Reading local image file...
Local image loaded and prepared.
Sending request to Gemini API...
Suc

# Editing using pandas


In [19]:
import pandas as pd
import json
from pathlib import Path

import re

exam_name = "2024_YKS_AYT"
# Load the JSON data from the file

json_path = Path("..") / "data" / f"{exam_name}_questions.json"

with open(json_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Convert the JSON data to a DataFrame

question_frame = pd.DataFrame(data["questions"])

# Add the image description to the questions

# loop over the questions and add the image description to the question text and also loop question choices and add the image description to the question choices

images_json = Path("..") / "data" / f"{exam_name}_images.json"

with open(images_json, "r", encoding="utf-8") as json_file:
    images_data = json.load(json_file)

image_frame = pd.DataFrame(images_data["images"])

for index, row in question_frame.iterrows():

    # Get the section and page number for the current question
    section = row["section"]
    page = row["page"]
    question_number = row["question_number"]

    print(f"Processing question {question_number} in section {section} on page {page}")

    
    # get the images on the same page
    images_on_page = image_frame[(image_frame["section"] == section) & (image_frame["page"] == page)]
    

    # Check if the row["question"] contains an image reference
    if "<img_" in row["question"]:
        # Loop through the image frame to find the matching image

        img_references = re.findall(r"<img_(\d+)\.(\w+)>", row["question"])


        for img_ref in img_references:
            img_number = int(img_ref[0])
            img_ext  = img_ref[1]

            # check if the image number is in the images_on_page
            if images_on_page[images_on_page["image_number"] == int(img_number)].shape[0] > 0:
                # Get the image description from the image frame
                print(f"Found image reference: {img_ref}")
                image_description = images_on_page[images_on_page["image_number"] == int(img_number)]["image_description"].values[0]

                print(f"Image description: {image_description}")

                # Replace the image reference in the question with the image description
                row["question"] = row["question"].replace(f"<img_{img_number}.{img_ext}>", image_description)

    
    # Loop through the choices and add the image description to the choices

    for ind, choice in enumerate(row["choices"]):
        if "<img_" in choice:
            # Loop through the image frame to find the matching image

            img_references = re.findall(r"<img_(\d+)\.(\w+)>", choice)

            for img_ref in img_references:
                img_number = int(img_ref[0])
                img_ext  = img_ref[1]

                # check if the image number is in the images_on_page
                if images_on_page[images_on_page["image_number"] == int(img_number)].shape[0] > 0:
                    # Get the image description from the image frame
                    image_description = images_on_page[images_on_page["image_number"] == int(img_number)]["image_description"].values[0]

                    # Replace the image reference in the question with the image description
                    choice = choice.replace(f"<img_{img_number}.{img_ext}>", image_description)

                    # Update the choice in the question frame

                    row["choices"][ind] = choice


    # Update the question frame with the modified question and choices

    question_frame.at[index, "question"] = row["question"]
    question_frame.at[index, "choices"] = row["choices"]





# Save the updated question frame to a JSON file

question_json = {
    "exam_year": data["exam_year"],
    "exam_type": data["exam_type"],
    "exam_subtype": data["exam_subtype"],
    "sections": data["sections"],
    "questions": question_frame.to_dict(orient="records"),
}

# Save the JSON data to a file
with open(f"../data/{exam_name}_questions_img_emb.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(question_json, ensure_ascii=False, indent=4))



Processing question 1 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 2
Processing question 2 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 2
Processing question 3 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 2
Processing question 4 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 3
Processing question 5 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 3
Processing question 6 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 4
Processing question 7 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 4
Processing question 8 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 4
Processing question 9 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 4
Processing question 10 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 5
Processing question 11 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 5
Processing question 12 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 5
Processing question 13 in section TURK-DILI-VE-EDEBIYATI-SOSYAL on page 5
Processing question 14 in section TURK-DILI-VE-